# Average wofflin principles values for real and generated art and the correlation between Wofflin's principles and feature representation of rela and generated art

In [1]:
import pickle
import pandas as pd
import numpy as np
import os

from pathlib import Path
from scipy.io import loadmat
from sklearn.decomposition import PCA

from torch.utils.data import Dataset, DataLoader, SequentialSampler
import torchvision.transforms as transforms
from PIL import Image
import glob 
import torch

from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
from scipy.stats import pearsonr

def get_principal_components(all_feats, n_components, verbose=False):
    pca = PCA(n_components=n_components, svd_solver='full')
    feats = pca.fit_transform(all_feats)
    if verbose:
        print(pca.explained_variance_ratio_)
        print(np.cumsum(pca.explained_variance_ratio_))
    return feats, len(pca.explained_variance_ratio_)

def get_coefficients(feats, df, n_components, verbose=False):
    pc, components = get_principal_components(feats, n_components, verbose=verbose)
    
    components_to_return = components
    print(f"{components} components required")
    components = min(components, 30)
    coefficients = {}
    principles = ['Linearly-vs-Painterly', 'Planar-vs-Recessional',
       'closed-form-vs-open-form', 'multiplicity-vs-unity',
       'absolute-clarity-vs-relative-clarity']
    
    for p in principles:
        for c in range(components):
            coefficients.setdefault(p,[]).append(pearsonr(df[p].values, pc[:, c])[0])
        
    return pd.DataFrame(coefficients), components_to_return

# Wofflin Average values for Real Art

In [3]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [4]:
df_1k = pd.read_csv('./datasets/processed/combined_wofflins_real.csv')
df_5k = pd.read_csv('./datasets/processed/real_wofflin_scores_combined_normalised.csv')
combined = [df_1k, df_5k]
df = pd.concat(combined)

In [5]:
for col in df.columns[1:6]:
    print(col,': ',df[col].mean(),'  ',df[col].std())    

Linearly-vs-Painterly :  0.6117693333333333    0.22268873157709945
Planar-vs-Recessional :  0.5237591666666667    0.19096761052776465
closed-form-vs-open-form :  0.5963766666666667    0.2184055500354099
multiplicity-vs-unity :  0.5555358333333333    0.21388193151692467
absolute-clarity-vs-relative-clarity :  0.5840611666666667    0.28601230847419606


# Correlation for real art

In [6]:
df['name'] = df['Input.image'].apply(lambda x: Path(x).name)
mat_file = loadmat('datasets/processed/groundtruth_pruned.mat')
files = [Path(f[0][0]).name for f in mat_file['groundtruth_pruned'][0][0][0]]
real_indexes = [files.index(f) for f in df.name.values]

In [7]:
folder = './datasets/features/real_art_features'
principles = ['Linearly-vs-Painterly', 'Planar-vs-Recessional',
       'closed-form-vs-open-form', 'multiplicity-vs-unity',
       'absolute-clarity-vs-relative-clarity']
table_for_n_components = {}
dataframes = {}
for file in os.listdir(folder):
    print(file)
    with open(os.path.join(folder, file), 'rb') as f:
        feats = pickle.load(f)
      
    coefficient_df, c = get_coefficients(feats[real_indexes], df, 0.95, verbose=False)
    table_for_n_components[file] = c
    dataframes[file] = coefficient_df.transpose()
    for p in range(5):
        sum=0
        maxi=0
        act_value=0
        index=0
        for i in range(coefficient_df.transpose().shape[1]):
            if(abs(coefficient_df.transpose()[i][p])>maxi):
                maxi = abs(coefficient_df.transpose()[i][p])
                act_value=coefficient_df.transpose()[i][p]
                index=i
            sum+=coefficient_df.transpose()[i][p]
        print(maxi,round(act_value,2),' -- ',index)
    print('##'*50)   

resnet101_plus2_real.pkl
25 components required
0.11157923480096395 -0.11  --  4
0.08530168210175196 -0.09  --  21
0.11118371381399887 -0.11  --  9
0.1033941818608744 0.1  --  5
0.15974494028616545 -0.16  --  9
####################################################################################################
resnet101_pretrained_real.pkl
372 components required
0.26825565263265605 0.27  --  6
0.2452351963981491 0.25  --  1
0.23820488019376934 0.24  --  1
0.37799687556290223 0.38  --  1
0.5390427438536964 0.54  --  1
####################################################################################################
resnet50_plus2_real.pkl
26 components required
0.17766250622188182 -0.18  --  2
0.07791598908279028 -0.08  --  2
0.08729875258289176 0.09  --  17
0.117349462137056 0.12  --  17
0.19065235832942462 0.19  --  17
####################################################################################################
resnet50_pretrained_real.pkl
395 components required
0.255232918

# Wofflin Average values and correlation for generated art

In [8]:
generations_csv = {
'StyleGAN2' : ('./datasets/features/generated_artworks/StyleGAN2/', './datasets/processed/combined_wofflins_styleGAN2.csv'),
'StyleCAN2' : ('./datasets/features/generated_artworks/StyleCAN2/', './datasets/processed/combined_wofflins_styleCAN2.csv'),
'StyleCWAN1' : ('./datasets/features/generated_artworks/StyleCWAN1/', './datasets/processed/combined_wofflins_styleCWAN1.csv'),
'StyleCWAN2' : ('./datasets/features/generated_artworks/StyleCWAN2/', './datasets/processed/combined_wofflins_styleCWAN2.csv'),
}

generated_indexes = {}
generated_csvs = {}

In [9]:
for k, (g, c) in generations_csv.items():
    print(k,g,c)
    items = glob.glob(g + '*')
    items = [i for i in items if 'lowest_shape_entropy' not in i]
    csv = pd.read_csv(c)
    names = list(csv['Input.image'].apply(lambda x: Path(x).name).values)
    indexes = [names.index(Path(i).name) for i in items]
    generated_csvs[k] = csv
    generated_indexes[k] = indexes

StyleGAN2 ./datasets/features/generated_artworks/StyleGAN2/ ./datasets/processed/combined_wofflins_styleGAN2.csv
StyleCAN2 ./datasets/features/generated_artworks/StyleCAN2/ ./datasets/processed/combined_wofflins_styleCAN2.csv
StyleCWAN1 ./datasets/features/generated_artworks/StyleCWAN1/ ./datasets/processed/combined_wofflins_styleCWAN1.csv
StyleCWAN2 ./datasets/features/generated_artworks/StyleCWAN2/ ./datasets/processed/combined_wofflins_styleCWAN2.csv


## Average wofflin's principle values for all generated art

In [10]:
for k, (g, c) in generations_csv.items():
    print(k)
    for col in generated_csvs[k].columns[1:6]:
        print(col,': ',generated_csvs[k][col].mean(),'   ',generated_csvs[k][col].std())    

StyleGAN2
multiplicity-vs-unity :  0.61475     0.1489139883954816
absolute-vs-relative :  0.7067500000000001     0.17220212376324348
linearly-vs-painterly :  0.7871250000000001     0.14441302635306527
planar-vs-recessional :  0.531875     0.17781811657267554
closed-form-vs-open-form :  0.626375     0.1371865874436568
StyleCAN2
absolute-vs-relative :  0.7007499999999999     0.1424787281313418
multiplicity-vs-unity :  0.651875     0.11709831088033494
linearly-vs-painterly :  0.801875     0.1500717768954289
planar-vs-recessional :  0.54375     0.14917798993720308
closed-form-vs-open-form :  0.6048749999999999     0.13119838763221608
StyleCWAN1
Linearly-vs-Painterly :  0.66375     0.14782783964950105
Planar-vs-Recessional :  0.6572500000000001     0.13464866563726036
closed-form-vs-open-form :  0.5898749999999999     0.15128102738982274
multiplicity-vs-unity :  0.5912299999999999     0.14341082162583998
absolute-clarity-vs-relative-clarity :  0.62375     0.13010863977985246
StyleCWAN2
Line

# Correlation Wofflin Principles and feature representation for generated art

In [11]:
folder = './datasets/features/generated_art_features/'

In [13]:
table_for_n_components = {}
dataframes = {}
for file in os.listdir(folder):
#     if file.split('_')[0] != 'vit':
#         print(file)
#         continue
    generation_name = file.split('_')[-1].split('.')[0]
    generation_model = file.split('_')[0]
    if generation_name in ['StyleGAN1','StyleCAN1']:
        continue
    print(generation_name)
    with open(os.path.join(folder, file), 'rb') as f:
        feats = pickle.load(f)
    generated_csvs[generation_name].columns = ['Input.image', 'multiplicity-vs-unity', 'absolute-clarity-vs-relative-clarity', 'Linearly-vs-Painterly', 'Planar-vs-Recessional',
       'closed-form-vs-open-form']
    coefficient_df, c = get_coefficients(feats, generated_csvs[generation_name], 0.95)
    table_for_n_components[file] = c
    dataframes[file] = coefficient_df.transpose()
    print(file)
    for p in range(5):
        sum=0
        maxi=0
        act_value=0
        index=0
        for i in range(coefficient_df.transpose().shape[1]):
            if(abs(coefficient_df.transpose()[i][p])>maxi):
                maxi=abs(coefficient_df.transpose()[i][p])
                act_value=coefficient_df.transpose()[i][p]
                index=i
            sum+=coefficient_df.transpose()[i][p]
        print(maxi,round(act_value,2),' -- ',index)
        
        print(maxi)
    print('##'*50)
#     break

StyleCAN2
25 components required
resnet101_plus2_StyleCAN2.pkl
0.2744455244964199 -0.27  --  3
0.2744455244964199
0.20855150553320329 0.21  --  17
0.20855150553320329
0.14220523655419237 0.14  --  17
0.14220523655419237
0.24011227071358104 -0.24  --  8
0.24011227071358104
0.14067748350562403 0.14  --  17
0.14067748350562403
####################################################################################################
StyleCWAN1
24 components required
resnet101_plus2_StyleCWAN1.pkl
0.09970147151876183 0.1  --  17
0.09970147151876183
0.11575958129139964 -0.12  --  19
0.11575958129139964
0.10438736036261576 -0.1  --  13
0.10438736036261576
0.08188692835877884 0.08  --  0
0.08188692835877884
0.1035110052072771 0.1  --  2
0.1035110052072771
####################################################################################################
StyleCWAN2
23 components required
resnet101_plus2_StyleCWAN2.pkl
0.12488479953288992 -0.12  --  4
0.12488479953288992
0.1354967935708567 -0.14  -- 

272 components required
StyleCAN2_StyleCAN2.pkl
0.28713004651762597 -0.29  --  0
0.28713004651762597
0.263554543873402 -0.26  --  0
0.263554543873402
0.26471659656825774 -0.26  --  0
0.26471659656825774
0.4694385595038453 -0.47  --  0
0.4694385595038453
0.24064051307394446 -0.24  --  3
0.24064051307394446
####################################################################################################
StyleCWAN1
286 components required
StyleCAN2_StyleCWAN1.pkl
0.13987146767786646 0.14  --  12
0.13987146767786646
0.15908665970336477 0.16  --  7
0.15908665970336477
0.10757183949427418 0.11  --  3
0.10757183949427418
0.10036635466731703 -0.1  --  1
0.10036635466731703
0.1124818247995795 -0.11  --  11
0.1124818247995795
####################################################################################################
StyleCWAN2
256 components required
StyleCAN2_StyleCWAN2.pkl
0.20943893265888666 0.21  --  6
0.20943893265888666
0.12074140935539182 -0.12  --  28
0.12074140935539182
0.23

14 components required
vgg16_plus2_StyleCWAN1.pkl
0.11864358073701672 -0.12  --  5
0.11864358073701672
0.11814530578860313 0.12  --  0
0.11814530578860313
0.11024438759488155 -0.11  --  10
0.11024438759488155
0.10502673167954964 0.11  --  12
0.10502673167954964
0.10225205158918513 -0.1  --  8
0.10225205158918513
####################################################################################################
StyleCWAN2
14 components required
vgg16_plus2_StyleCWAN2.pkl
0.1271095292063074 -0.13  --  5
0.1271095292063074
0.13339431998352916 -0.13  --  7
0.13339431998352916
0.15826885545681152 -0.16  --  3
0.15826885545681152
0.1291748477935697 -0.13  --  4
0.1291748477935697
0.12749556227140973 -0.13  --  7
0.12749556227140973
####################################################################################################
StyleGAN2
15 components required
vgg16_plus2_StyleGAN2.pkl
0.18063181132564674 -0.18  --  3
0.18063181132564674
0.169034747519004 -0.17  --  4
0.169034747519004
0

# To visualise the Wolfflin Principles

### We toggle between asc=False and asc=True, to view the extremes of pair of Wofflin Principles, and different cols indexes are used for different principles.

In [14]:
acs = False

In [15]:
cols  = ['Linearly-vs-Painterly','Planar-vs-Recessional','closed-form-vs-open-form','multiplicity-vs-unity','absolute-clarity-vs-relative-clarity']
col = cols[0] 
name  = 'Input.image'
generated_csvs['StyleCWAN2'][[col,name]].groupby(name).mean().sort_values(by=col,ascending=acs)

,Linearly-vs-Painterly
Input.image,
https://crw2020.s3-us-west-2.amazonaws.com/sg2/RW-W5_human_evaluation/highest_NN_dist000805.png,0.95
https://crw2020.s3-us-west-2.amazonaws.com/sg2/RW-W5_human_evaluation/lowest_NN_dist000986.png,0.95
https://crw2020.s3-us-west-2.amazonaws.com/sg2/RW-W5_human_evaluation/lowest_NN_dist000015.png,0.95
https://crw2020.s3-us-west-2.amazonaws.com/sg2/RW-W5_human_evaluation/lowest_NN_dist000221.png,0.90
https://crw2020.s3-us-west-2.amazonaws.com/sg2/RW-W5_human_evaluation/highest_NN_dist000963.png,0.90
https://crw2020.s3-us-west-2.amazonaws.com/sg2/RW-W5_human_evaluation/highest_NN_dist000574.png,0.90
https://crw2020.s3-us-west-2.amazonaws.com/sg2/RW-W5_human_evaluation/highest_shape_entropy000694.png,0.90
https://crw2020.s3-us-west-2.amazonaws.com/sg2/RW-W5_human_evaluation/random000046.png,0.85
https://crw2020.s3-us-west-2.amazonaws.com/sg2/RW-W5_human_evaluation/highest_shape_entropy000675.png,0.85
